<a href="https://colab.research.google.com/github/juliawol/WB_Embedder/blob/main/Brand_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets sentence-transformers tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm


# Configuration
data_files = "data_sampled_30.csv"
model_name = "DeepPavlov/rubert-base-cased"  # Pre-trained RuBERT model for embeddings
threshold = 0.7  # Similarity threshold for considering brand pairs as candidates

# Load dataset using the datasets library
dataset = load_dataset("JuliaWolken/WB_CARDS", data_files=data_files)
data = dataset["train"].to_pandas()

# Filter necessary rows and columns
data = data.dropna(subset=['brandname'])

# Brand Candidate Generation
def generate_brand_candidates(data, model_name, threshold):
    # Load pre-trained model for embeddings
    model = SentenceTransformer(model_name)

    # Get unique brand names
    brands = data['brandname'].dropna().unique()

    # Compute embeddings for all brands
    print("Generating embeddings for brands...")
    brand_embeddings = model.encode(brands, convert_to_tensor=True, show_progress_bar=True)

    # Find similar brand pairs using cosine similarity
    print("Finding similar brand pairs...")
    candidates = []
    similarities = util.pytorch_cos_sim(brand_embeddings, brand_embeddings)

    for i in tqdm(range(len(brands)), desc="Processing brands"):
        for j in range(i + 1, len(brands)):
            similarity = similarities[i, j].item()
            if similarity >= threshold:
                candidates.append((brands[i], brands[j], "Positive"))

    return candidates

# Generate brand candidates
brand_candidates = generate_brand_candidates(data, model_name, threshold)

# Save candidates for review with progress bar
print("Saving brand candidates to CSV...")
with tqdm(total=len(brand_candidates), desc="Saving to CSV") as pbar:
    brand_candidates_df = pd.DataFrame(brand_candidates, columns=['Brand1', 'Brand2', 'Label'])
    brand_candidates_df.to_csv("brand_candidates.csv", index=False)
    pbar.update(len(brand_candidates))

print("Brand candidate generation complete. File saved as 'brand_candidates.csv'.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


data_sampled_30.csv:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

...

Finding similar brand pairs...


Processing brands: 100%|██████████| 35681/35681 [2:48:07<00:00,  3.54it/s] 


Saving brand candidates to CSV...


Saving to CSV: 100%|██████████| 22735953/22735953 [00:24<00:00, 925745.24it/s]

Brand candidate generation complete. File saved as 'brand_candidates.csv'.
